# **Energy Consumption & CO2 Emissions Analysis**

## Hypothesis Testing

We'll test several hypotheses about energy consumption and CO2 emissions.

### Hypothesis 1: GDP and Energy Consumption

**H0 (Null):** There is no significant correlation between GDP and energy consumption

**H1 (Alternative):** There is a significant positive correlation between GDP and energy consumption

**Significance Level:** α = 0.05